In [ ]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from datasets import load_dataset

# Step 1: Load the dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:1%]")  # use only 1% for simplicity


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:

# Step 2: Initialize Tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize_function(example):
    return tokenizer(example['text'], truncation=True, padding="max_length", max_length=50)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Step 3: Define Dataset and DataLoader for Language Modeling
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_datasets, seq_len=5):
        self.seq_len = seq_len
        self.input_ids = tokenized_datasets['input_ids']

    def __len__(self):
        return sum(len(line) - self.seq_len for line in self.input_ids)

    def __getitem__(self, idx):
        line = self.input_ids[idx // (len(self.input_ids) - self.seq_len)]
        x = line[idx % (len(line) - self.seq_len): idx % (len(line) - self.seq_len) + self.seq_len]
        y = line[idx % (len(line) - self.seq_len) + self.seq_len]
        return x, y

dataset = TextDataset(tokenized_datasets)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
# Step 4: Define the Simple Transformer model
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, embed_size=32, num_heads=2, num_layers=2, max_seq_len=50):
        super(SimpleTransformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.position_encoding = nn.Parameter(torch.randn(max_seq_len, embed_size))  # Max sequence length
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embed_size, num_heads), num_layers
        )
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, x):
        seq_len = x.size(1)  # Get the current sequence length
        position_encoding = self.position_encoding[:seq_len, :].unsqueeze(0)  # Adjust position encoding dynamically
        x = self.embedding(x) + position_encoding  # Add position encoding to embeddings
        x = self.transformer(x.transpose(0, 1)).transpose(0, 1)  # Transpose for transformer
        x = self.fc(x[:, -1])  # Predict the next word (last position)
        return x



In [ ]:

# Model, optimizer, and loss function
model = SimpleTransformer(vocab_size=tokenizer.vocab_size)
optimizer = Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# Step 5: Train the Model
for epoch in range(5):  # Fewer epochs for demonstration
    total_loss = 0
    for x, y in data_loader:
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(data_loader):.4f}")

Epoch 1, Loss: 2.2784
Epoch 2, Loss: 1.2739
Epoch 3, Loss: 0.7314
Epoch 4, Loss: 0.4516
Epoch 5, Loss: 0.3190


In [ ]:
def generate_text(model, tokenizer, start_text, max_len=10):
    model.eval()

    # Tokenize the input text
    tokens = tokenizer.encode(start_text, return_tensors='pt')

    generated = tokens.clone()

    for _ in range(max_len):
        with torch.no_grad():
            output = model(generated)
            next_token = torch.argmax(output, dim=-1)[-1].unsqueeze(0)  # Get the last predicted token
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)  # Append the new token

    return tokenizer.decode(generated.squeeze().tolist())

# Example usage
print("Generated Text:", generate_text(model, tokenizer, "the quick brown"))


Generated Text: [CLS] the quick brown [SEP] [PAD] valkyria chronicles missions ( [PAD] [PAD] [PAD]
